# Run simulations - INCLINE

This notebook goes through the steps of setting up cases and running simulations for the paper "..." 
Simulations were run using the [NorESM Landsites Platform](https://noresmhub.github.io/NorESM_LandSites_Platform)(NLP). See the [technical documentation](https://noresmhub.github.io/NorESM_LandSites_Platform) and [user guide](https://noresmhub.github.io/NorESM_LandSites_Platform/user_guide) for further details.

Run on NREC using the Docker container from there?

- NorESM landsites platform version
- NorESM landsites platform input data version
- NorESM version
- CLM version
- FATES version

### Workflow

0. Set up modelling software, container etc., for simulations
1. Prepare input data (separate notebook)
2. Make parameter files (separate notebook)
3. Run single-PFT simulations
4. Run transplant experiment baseline
5. Run transplant experiment with warming
6. Analyse output (separate notebook)

## 0. Set up modelling software

Use the NLP on NREC. 
Log in to NREC 

    ssh -i .ssh/evaskey centos@158.39.201.12



## 1. [Prepare inputdata]() (separate notebook)

## 2. [Make parameter files]() (separate notebook)


## 3. Set up and run single-PFT simulations

To test whether our nine custom PFTs survive and grow at all, we first need to run single-PFT simulations for the four study sites = 36 simulations.

Nine PFTs: 
sib_pro_min, 
sib_pro_mean,
sib_pro_max,
suc_pra_min, 
suc_pra_mean,
suc_pra_max, 
ver_off_min, 
ver_off_mean,
ver_off_max 

Four sites: ALP1, ALP2, ALP3, ALP4.

Parameter file ...
Compset ...
Settings file ...
Model run type

docker-compose up
upload this notebook
run this notebook

In [ ]:
! cp ./landsites_tools/settings.txt ./landsites_tools/custom_settings/settings1.txt

! vi landsites_tools/custom_settings/settings1.txt
# add sites ALP1-4; make changes using i and esc, :wq

In [ ]:
# make cases
!./landsites_tools/simulation/make_cases.sh -sfxc _sib_s -f settings1.txt
!./landsites_tools/simulation/make_cases.sh -sfxc _sib_m -f settings1.txt
!./landsites_tools/simulation/make_cases.sh -sfxc _sib_l -f settings1.txt
!./landsites_tools/simulation/make_cases.sh -sfxc _suc_s -f settings1.txt
!./landsites_tools/simulation/make_cases.sh -sfxc _suc_m -f settings1.txt
!./landsites_tools/simulation/make_cases.sh -sfxc _suc_l -f settings1.txt
!./landsites_tools/simulation/make_cases.sh -sfxc _ver_s -f settings1.txt
!./landsites_tools/simulation/make_cases.sh -sfxc _ver_m -f settings1.txt
!./landsites_tools/simulation/make_cases.sh -sfxc _ver_l -f settings1.txt


The custom parameter files need to be specified in 

In [ ]:
%%bash
# set path to parameter files
fates_default_param_path=/home/centos/NorESM_LandSites_Platform/data/input/clm/ALP1_0.1.0_input/inputdata/lnd/clm2/paramdata/
fates_default_param_fname=fates_params_api.14.0.0_12pft_c200921
fates_path=/home/centos/NorESM_LandSites_Platform/noresm2/components/clm/src/fates/

ncdump ${fates_default_param_path}${fates_default_param_fname}.nc > ${fates_default_param_path}${fates_default_param_fname}.cdl
python3 ${fates_path}tools/FatesPFTIndexSwapper.py --pft-indices 10 --fin ${fates_default_param_path}${fates_default_param_fname}.cdl --fout ${fates_path}parameter_files/fates_params_sib_m.cdl

# problem! use winscp to replace parameterfile with a new one.

Simulation period is set in the env_run.xml file in the data/cases/casename folder

In [ ]:
# change namelist options
#case_names=(ALP1_0.1.0_sib_m ALP2_0.1.0_sib_m ALP3_0.1.0_sib_m ALP4_0.1.0_sib_m)

for case_name in ALP1_0.1.0_sib_m ALP2_0.1.0_sib_m ALP3_0.1.0_sib_m ALP4_0.1.0_sib_m
do
 cd /home/centos/NorESM_LandSites_Platform/data/cases/${case_name}

#loop to change for all the cases

./xmlchange --file env_run.xml --id RUN_STARTDATE --val 2000-01-01                
./xmlchange --file env_run.xml --id STOP_OPTION --val nyears               
./xmlchange --file env_run.xml --id STOP_N --val 9                        
./xmlchange --file env_run.xml --id DOUT_S --val TRUE 
#./xmlchange --file env_run.xml --id DIN_LOC_ROOT_CLMFORC --val *****${input_dir}/***
./xmlchange --file env_run.xml --id DATM_CLMNCEP_YR_ALIGN --val 2000
./xmlchange --file env_run.xml --id DATM_CLMNCEP_YR_START --val 2000
./xmlchange --file env_run.xml --id DATM_CLMNCEP_YR_END --val 2009   
# ./xmlchange --file env_run.xml --id LND_TUNING_MODE --val clm5_0_CRUv7   
done


In [ ]:

# run cases
!./landsites_tools/simulation/make_cases.sh -f settings_.txt

## 4. Run transplant experiment with current climate

Changes to settings file:
Sites ALP1, ALP2, ALP3, ALP4.
Model run type: 





In [ ]:
# make cases with custom settings file
!python ../landsites_tools/simulation/make_cases.py -f settings_exp_c.txt

In [ ]:
# run cases
!./landsites_tools/simulation/run_cases.sh -f settings_exp.txt -q

After the run terminates, output is in `data/output`. Make sure the container is up until the simulation has finished. See separate notebook for output processing and plotting.

## 5. Run transplant experiment with OTC warming

Use the same parameter files as before, but change temperature (how?) to 2 degrees C warmer ... 

In [ ]:
!./landsites_tools/simulation/make_cases.sh -f settings_exp.txt

In [ ]:
!./landsites_tools/simulation/run_cases.sh -f settings_exp.txt -q